# Predicting Tennis Match Results: Data Wrangling

## Notebook by Lauren Dellon

## License

This dataset was exported from Kaggle, which used Jeff Sackmann's GitHub Contribution under CC attributions, non-commercial, and share alike license.

## Dataset Description

The dataset comes from Kaggle: \
https://www.kaggle.com/pablodroca/atp-tennis-matches-20002019?select=atp_matches_2000.csv

The dataset includes 21 individual csv files: \
1 csv file for matches in each year from 2000-2019 \
1 csv file for players

The matches csv files have the following columns: \
**tourney_name**: a unique identifier for each tournament \
**tourney_date**: date of the tournament in YYYYMMDD format \
**surface**: court surface (hard, clay, etc.) \
**winner_id**: player_id for winner of the match \
**loser_id**: player_id for loser of the match \
**score**: match score \
**best_of**: '3' or '5', indicating the maximum number of sets for the match \
**round**: round of tournament \
**minutes**: match time in minutes \
**w_ace, l_ace**: number of aces for winner, loser \
**w_df, l_df**: number of double faults for winner, loser \
**w_svpt, l_svpt**: number of serve points for winner, loser \
**w_1stIn, l_1stIn**: number of first serves made for winner, loser \
**w_1stWon, l_1stWon**: number of first-serve points won for winner, loser \
**w_2ndWon, l_2ndWon**: number of second-serve points won for winner, loser \
**w_SvGms, l_SvGms**: number of serve games for winner, loser \
**w_bpSaved, l_bpSaved**: number of break points saved for winner, loser \
**w_bpFaced, l_bpFaced**: number of break points faced for winner, loser \
**winner_rank, loser_rank**: winner's and loser's rank, as of the tourney_date \
**winner_rank_points, loser_rank_point**: winner's and loser's number of ranking points, as of the tourney date \

The players csv file has the following columns: \
**player_id**: a unique identifier for each player \
**name_first**: first name of player \
**name_last**: last name of player \
**hand**: 'R', 'L', 'A', or 'U' for right-handed, left-handed, ambidextrous, or unsure \
**birthdate**: birthdate of player \
**country**: player's country 

## Imports and Reading Files

In [1]:
# First, import the relevant modules and packages
import pandas as pd
import numpy as np
import glob
from datetime import date
import matplotlib.pyplot as plt
from library.sb_utils import save_file

In [2]:
# Get csv file names
file_names = glob.glob('../raw_data/atp_matches*.csv')

In [3]:
# Read in atp_matches files and create DataFrame with all data
ls = []
for file_name in file_names:
    ls.append(pd.read_csv(file_name))
df_matches = pd.concat(ls, ignore_index=True)

In [4]:
df_matches.head()

,tourney_id,tourney_name,tourney_date,surface,winner_id,loser_id,score,best_of,round,minutes,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2019-M020,Brisbane,20181231,Hard,106045,126203,7-6(5) 6-7(2) 6-4,3,R32,144.0,...,72.0,57.0,27.0,17.0,9.0,10.0,63.0,810.0,49.0,974.0
1,2019-M020,Brisbane,20181231,Hard,105357,105815,7-6(6) 6-7(6) 6-0,3,R32,153.0,...,63.0,44.0,20.0,15.0,13.0,18.0,38.0,1083.0,61.0,814.0
2,2019-M020,Brisbane,20181231,Hard,105777,106415,6-3 6-4,3,R32,79.0,...,33.0,21.0,7.0,9.0,0.0,3.0,19.0,1835.0,75.0,701.0
3,2019-M020,Brisbane,20181231,Hard,106034,200005,6-4 7-6(6),3,R32,90.0,...,45.0,35.0,14.0,11.0,0.0,1.0,185.0,275.0,102.0,572.0
4,2019-M020,Brisbane,20181231,Hard,104871,105526,4-6 6-3 6-4,3,R32,99.0,...,41.0,32.0,22.0,14.0,4.0,6.0,40.0,1050.0,57.0,875.0


In [5]:
# See summary of the data
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59430 entries, 0 to 59429
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          59430 non-null  object 
 1   tourney_name        59430 non-null  object 
 2   tourney_date        59430 non-null  int64  
 3   surface             59312 non-null  object 
 4   winner_id           59430 non-null  int64  
 5   loser_id            59430 non-null  int64  
 6   score               59429 non-null  object 
 7   best_of             59430 non-null  int64  
 8   round               59430 non-null  object 
 9   minutes             52431 non-null  float64
 10  w_ace               53749 non-null  float64
 11  w_df                53749 non-null  float64
 12  w_svpt              53749 non-null  float64
 13  w_1stIn             53749 non-null  float64
 14  w_1stWon            53749 non-null  float64
 15  w_2ndWon            53749 non-null  float64
 16  w_Sv

The df_matches dataframe has 59430 entries and 32 columns.

## Missing Values

In [6]:
# Count the number of missing values in each column and sort them
missing = pd.concat([df_matches.isnull().sum(), 100 * df_matches.isnull().mean()], axis=1)
missing.columns = ['count','%']
missing.sort_values(by='count')

,count,%
tourney_id,0,0.000000
tourney_name,0,0.000000
tourney_date,0,0.000000
winner_id,0,0.000000
loser_id,0,0.000000
best_of,0,0.000000
round,0,0.000000
score,1,0.001683
surface,118,0.198553
winner_rank_points,498,0.837961


For most of the stats columns, there are 5681 rows with missing data. Let's investigate.

In [7]:
print(df_matches[df_matches['w_1stWon'].isnull()])

      tourney_id                tourney_name  tourney_date surface  winner_id  \
45     2019-0451                        Doha      20181231    Hard     106065   
383    2019-7434                       Sofia      20190204    Hard     105916   
560    2019-6932              Rio De Janeiro      20190218    Clay     111513   
579    2019-M004                    Acapulco      20190225    Hard     200282   
707    2018-0451                        Doha      20180101    Hard     104792   
...          ...                         ...           ...     ...        ...   
59353   2014-573                    Valencia      20141020    Hard     104731   
59426   2014-605                 Tour Finals      20141109    Hard     104925   
59427  2014-D015  Davis Cup WG F: FRA vs SUI      20141121    Clay     104527   
59428  2014-D015  Davis Cup WG F: FRA vs SUI      20141121    Clay     104792   
59429  2014-D015  Davis Cup WG F: FRA vs SUI      20141121    Clay     103819   

       loser_id            

When one stat is missing, the others appear to be missing as well.

Before removing these missing values, let's drop columns with irrelevant information: tourney_id and tourney_name.
Also, drop columns that are the same regardless of winner or loser: surface, score, round, and minutes

In [8]:
df_matches.drop(columns=['tourney_id','tourney_name','surface','score', \
                        'round','minutes'], inplace=True)

In [9]:
# Drop rows with missing stats data
df_matches.dropna(subset=['w_1stWon','winner_rank','loser_rank'], inplace=True)

In [10]:
# Count the number of missing values in each column and sort them
missing = pd.concat([df_matches.isnull().sum(), 100 * df_matches.isnull().mean()], axis=1)
missing.columns = ['count','%']
missing.sort_values(by='count')

,count,%
tourney_date,0,0.0
winner_rank_points,0,0.0
winner_rank,0,0.0
l_bpFaced,0,0.0
l_bpSaved,0,0.0
l_SvGms,0,0.0
l_2ndWon,0,0.0
l_1stWon,0,0.0
l_1stIn,0,0.0
l_svpt,0,0.0


Great, no more missing data.

In [11]:
# Tennis matches can be either best of 5 sets or best of 3 sets. See how many of each there are in our dataset.
df_matches['best_of'].value_counts()

3    43320
5    10152
Name: best_of, dtype: int64

In [12]:
# We need to drop either the rows with best of 5 or the rows with best of 3 because all the stats are absolute
# numbers, so they would obviously be higher for matches with 5 sets which would skew the results.
# So, drop the rows with a best of 5. Then, drop the best_of column.
df_matches = df_matches[df_matches.best_of == 3]
df_matches.drop(columns=['best_of'], inplace=True)

In [13]:
# Convert tourney_date to datetime object
df_matches['tourney_date'] = df_matches['tourney_date'].astype(str)
df_matches['tourney_date'] = pd.to_datetime(df_matches['tourney_date'], format = '%Y%m%d')

In [14]:
df_matches.head()

,tourney_date,winner_id,loser_id,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2018-12-31,106045,126203,8.0,4.0,117.0,73.0,58.0,26.0,17.0,...,72.0,57.0,27.0,17.0,9.0,10.0,63.0,810.0,49.0,974.0
1,2018-12-31,105357,105815,8.0,2.0,98.0,68.0,49.0,15.0,15.0,...,63.0,44.0,20.0,15.0,13.0,18.0,38.0,1083.0,61.0,814.0
2,2018-12-31,105777,106415,9.0,6.0,76.0,43.0,37.0,12.0,10.0,...,33.0,21.0,7.0,9.0,0.0,3.0,19.0,1835.0,75.0,701.0
3,2018-12-31,106034,200005,12.0,0.0,58.0,43.0,39.0,10.0,11.0,...,45.0,35.0,14.0,11.0,0.0,1.0,185.0,275.0,102.0,572.0
4,2018-12-31,104871,105526,15.0,4.0,87.0,52.0,40.0,21.0,15.0,...,41.0,32.0,22.0,14.0,4.0,6.0,40.0,1050.0,57.0,875.0


## Separate matches into wins and losses

In [15]:
# Separate the matches into wins and losses
df_wins = df_matches[['tourney_date','winner_id','winner_rank_points','winner_rank','w_bpFaced','w_bpSaved',\
                      'w_SvGms','w_2ndWon','w_1stWon','w_1stIn','w_svpt','w_df','w_ace']]

df_losses = df_matches[['tourney_date','loser_id','loser_rank_points','loser_rank','l_bpFaced','l_bpSaved',\
                        'l_SvGms','l_2ndWon','l_1stWon','l_1stIn','l_svpt','l_df','l_ace']]

In [16]:
# Add a column for outcome
df_wins.loc[:,'outcome'] = 1
df_losses.loc[:,'outcome'] = 0

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [17]:
df_wins.head()

,tourney_date,winner_id,winner_rank_points,winner_rank,w_bpFaced,w_bpSaved,w_SvGms,w_2ndWon,w_1stWon,w_1stIn,w_svpt,w_df,w_ace,outcome
0,2018-12-31,106045,810.0,63.0,4.0,4.0,17.0,26.0,58.0,73.0,117.0,4.0,8.0,1
1,2018-12-31,105357,1083.0,38.0,5.0,3.0,15.0,15.0,49.0,68.0,98.0,2.0,8.0,1
2,2018-12-31,105777,1835.0,19.0,7.0,6.0,10.0,12.0,37.0,43.0,76.0,6.0,9.0,1
3,2018-12-31,106034,275.0,185.0,0.0,0.0,11.0,10.0,39.0,43.0,58.0,0.0,12.0,1
4,2018-12-31,104871,1050.0,40.0,3.0,2.0,15.0,21.0,40.0,52.0,87.0,4.0,15.0,1


In [18]:
df_losses.head()

,tourney_date,loser_id,loser_rank_points,loser_rank,l_bpFaced,l_bpSaved,l_SvGms,l_2ndWon,l_1stWon,l_1stIn,l_svpt,l_df,l_ace,outcome
0,2018-12-31,126203,974.0,49.0,10.0,9.0,17.0,27.0,57.0,72.0,123.0,4.0,22.0,0
1,2018-12-31,105815,814.0,61.0,18.0,13.0,15.0,20.0,44.0,63.0,106.0,4.0,12.0,0
2,2018-12-31,106415,701.0,75.0,3.0,0.0,9.0,7.0,21.0,33.0,47.0,1.0,1.0,0
3,2018-12-31,200005,572.0,102.0,1.0,0.0,11.0,14.0,35.0,45.0,75.0,4.0,6.0,0
4,2018-12-31,105526,875.0,57.0,6.0,4.0,14.0,22.0,32.0,41.0,74.0,2.0,10.0,0


In [19]:
# Rename the columns so they are identical for df_wins and df_losses
df_wins = df_wins.rename(columns={'winner_id':'player_id','winner_rank_points':'rank_points','winner_rank':'rank',\
                       'w_bpFaced':'bpFaced','w_bpSaved':'bpSaved','w_SvGms':'SvGms','w_2ndWon':'2ndWon',\
                       'w_1stWon':'1stWon','w_1stIn':'1stIn','w_svpt':'svpt','w_df':'df','w_ace':'ace'})
df_losses = df_losses.rename(columns={'loser_id':'player_id','loser_rank_points':'rank_points','loser_rank':'rank',\
                       'l_bpFaced':'bpFaced','l_bpSaved':'bpSaved','l_SvGms':'SvGms','l_2ndWon':'2ndWon',\
                       'l_1stWon':'1stWon','l_1stIn':'1stIn','l_svpt':'svpt','l_df':'df','l_ace':'ace'})

## Supplement with player data

In [20]:
# First, read in the player data
df_players = pd.read_csv('../raw_data/atp_players.csv')
df_players.head()

,player_id,name_first,name_list,hand,birthdate,country
0,100001,Gardnar,Mulloy,R,19131122.0,USA
1,100002,Pancho,Segura,R,19210620.0,ECU
2,100003,Frank,Sedgman,R,19271002.0,AUS
3,100004,Giuseppe,Merlo,R,19271011.0,ITA
4,100005,Richard Pancho,Gonzales,R,19280509.0,USA


In [21]:
# Clean up a bit and drop irrelevant columns
df_players['name'] = df_players['name_first'] + ' ' + df_players['name_list']
df_players.drop(columns=['name_first','name_list','country'], inplace=True)

In [22]:
# Convert birthdate to datetime object
df_players['birthdate'] = df_players['birthdate'].astype(str)
df_players['birthdate'] = pd.to_datetime(df_players['birthdate'], format = '%Y%m%d')

In [23]:
df_wins = pd.merge(df_wins, df_players, how='left', on='player_id')
df_wins.head()

,tourney_date,player_id,rank_points,rank,bpFaced,bpSaved,SvGms,2ndWon,1stWon,1stIn,svpt,df,ace,outcome,hand,birthdate,name
0,2018-12-31,106045,810.0,63.0,4.0,4.0,17.0,26.0,58.0,73.0,117.0,4.0,8.0,1,R,1992-08-17,Denis Kudla
1,2018-12-31,105357,1083.0,38.0,5.0,3.0,15.0,15.0,49.0,68.0,98.0,2.0,8.0,1,R,1989-06-14,John Millman
2,2018-12-31,105777,1835.0,19.0,7.0,6.0,10.0,12.0,37.0,43.0,76.0,6.0,9.0,1,R,1991-05-16,Grigor Dimitrov
3,2018-12-31,106034,275.0,185.0,0.0,0.0,11.0,10.0,39.0,43.0,58.0,0.0,12.0,1,R,1992-08-05,Yasutaka Uchiyama
4,2018-12-31,104871,1050.0,40.0,3.0,2.0,15.0,21.0,40.0,52.0,87.0,4.0,15.0,1,R,1987-02-12,Jeremy Chardy


In [24]:
df_losses = pd.merge(df_losses, df_players, how='left', on='player_id')
df_losses.head()

,tourney_date,player_id,rank_points,rank,bpFaced,bpSaved,SvGms,2ndWon,1stWon,1stIn,svpt,df,ace,outcome,hand,birthdate,name
0,2018-12-31,126203,974.0,49.0,10.0,9.0,17.0,27.0,57.0,72.0,123.0,4.0,22.0,0,R,1997-10-28,Taylor Harry Fritz
1,2018-12-31,105815,814.0,61.0,18.0,13.0,15.0,20.0,44.0,63.0,106.0,4.0,12.0,0,R,1991-07-22,Tennys Sandgren
2,2018-12-31,106415,701.0,75.0,3.0,0.0,9.0,7.0,21.0,33.0,47.0,1.0,1.0,0,L,1995-09-27,Yoshihito Nishioka
3,2018-12-31,200005,572.0,102.0,1.0,0.0,11.0,14.0,35.0,45.0,75.0,4.0,6.0,0,L,1998-06-26,Ugo Humbert
4,2018-12-31,105526,875.0,57.0,6.0,4.0,14.0,22.0,32.0,41.0,74.0,2.0,10.0,0,R,1990-04-25,Jan Lennard Struff


In [25]:
# Concatenate the wins and losses into a single dataframe
df_all = pd.concat([df_wins,df_losses])
df_all.head()

,tourney_date,player_id,rank_points,rank,bpFaced,bpSaved,SvGms,2ndWon,1stWon,1stIn,svpt,df,ace,outcome,hand,birthdate,name
0,2018-12-31,106045,810.0,63.0,4.0,4.0,17.0,26.0,58.0,73.0,117.0,4.0,8.0,1,R,1992-08-17,Denis Kudla
1,2018-12-31,105357,1083.0,38.0,5.0,3.0,15.0,15.0,49.0,68.0,98.0,2.0,8.0,1,R,1989-06-14,John Millman
2,2018-12-31,105777,1835.0,19.0,7.0,6.0,10.0,12.0,37.0,43.0,76.0,6.0,9.0,1,R,1991-05-16,Grigor Dimitrov
3,2018-12-31,106034,275.0,185.0,0.0,0.0,11.0,10.0,39.0,43.0,58.0,0.0,12.0,1,R,1992-08-05,Yasutaka Uchiyama
4,2018-12-31,104871,1050.0,40.0,3.0,2.0,15.0,21.0,40.0,52.0,87.0,4.0,15.0,1,R,1987-02-12,Jeremy Chardy


## More cleaning

In [26]:
# Calculate age of player at time of tourney
df_all['age'] = df_all.tourney_date - df_all.birthdate
df_all['age'] = df_all.age / np.timedelta64(1, 'Y')
df_all['age'] = df_all['age'].apply(np.floor)

In [27]:
# Create new column with year of tourney, and then drop the tourney_date, player_id, and birthdate columns
df_all['tourney_year'] = pd.DatetimeIndex(df_all['tourney_date']).year
df_all.drop(columns=['tourney_date','player_id','birthdate'], inplace=True)

In [28]:
df_all.head()

,rank_points,rank,bpFaced,bpSaved,SvGms,2ndWon,1stWon,1stIn,svpt,df,ace,outcome,hand,name,age,tourney_year
0,810.0,63.0,4.0,4.0,17.0,26.0,58.0,73.0,117.0,4.0,8.0,1,R,Denis Kudla,26.0,2018
1,1083.0,38.0,5.0,3.0,15.0,15.0,49.0,68.0,98.0,2.0,8.0,1,R,John Millman,29.0,2018
2,1835.0,19.0,7.0,6.0,10.0,12.0,37.0,43.0,76.0,6.0,9.0,1,R,Grigor Dimitrov,27.0,2018
3,275.0,185.0,0.0,0.0,11.0,10.0,39.0,43.0,58.0,0.0,12.0,1,R,Yasutaka Uchiyama,26.0,2018
4,1050.0,40.0,3.0,2.0,15.0,21.0,40.0,52.0,87.0,4.0,15.0,1,R,Jeremy Chardy,31.0,2018


In [29]:
# Rearrange columns
df_all = df_all[['name','tourney_year','age','hand','rank_points','rank','bpFaced','bpSaved','SvGms','2ndWon',\
                 '1stWon','1stIn','svpt','df','ace','outcome']]

In [30]:
# Count the number of missing values in each column and sort them
missing = pd.concat([df_all.isnull().sum(), 100 * df_all.isnull().mean()], axis=1)
missing.columns = ['count','%']
missing.sort_values(by='count')

,count,%
name,0,0.000000
tourney_year,0,0.000000
rank_points,0,0.000000
rank,0,0.000000
bpFaced,0,0.000000
bpSaved,0,0.000000
SvGms,0,0.000000
2ndWon,0,0.000000
1stWon,0,0.000000
1stIn,0,0.000000


In [31]:
# Drop rows with missing stats data
df_all.dropna(subset=['age','hand'], inplace=True)

In [32]:
# What are the unique categories for 'hand'?
df_all['hand'].unique()

array(['R', 'L', 'U'], dtype=object)

In [33]:
df_all['hand'].value_counts()

R    75071
L    11159
U      401
Name: hand, dtype: int64

In [34]:
# Assuming 'U' stands for uncertain/unsure, drop these rows.
df_all = df_all[df_all.hand != 'U']

In [35]:
# Use get_dummies for the categorial variable 'hand'
df_all = pd.get_dummies(df_all, columns=['hand'])

In [36]:
df_all.head()

,name,tourney_year,age,rank_points,rank,bpFaced,bpSaved,SvGms,2ndWon,1stWon,1stIn,svpt,df,ace,outcome,hand_L,hand_R
0,Denis Kudla,2018,26.0,810.0,63.0,4.0,4.0,17.0,26.0,58.0,73.0,117.0,4.0,8.0,1,0,1
1,John Millman,2018,29.0,1083.0,38.0,5.0,3.0,15.0,15.0,49.0,68.0,98.0,2.0,8.0,1,0,1
2,Grigor Dimitrov,2018,27.0,1835.0,19.0,7.0,6.0,10.0,12.0,37.0,43.0,76.0,6.0,9.0,1,0,1
3,Yasutaka Uchiyama,2018,26.0,275.0,185.0,0.0,0.0,11.0,10.0,39.0,43.0,58.0,0.0,12.0,1,0,1
4,Jeremy Chardy,2018,31.0,1050.0,40.0,3.0,2.0,15.0,21.0,40.0,52.0,87.0,4.0,15.0,1,0,1


In [37]:
# Drop hand_L to remove collinearity
df_all = df_all.drop(columns=['hand_L'])

In [39]:
# Extract numerical columns into separate dataframe to explore
df_numeric = df_all[['age','rank_points','rank','bpFaced','bpSaved','SvGms','2ndWon','1stWon','1stIn',\
                    'svpt','df','ace','hand_R','outcome']]
df_numeric.describe().T

,count,mean,std,min,25%,50%,75%,max
age,86230.0,25.918416,3.808094,15.0,23.0,26.0,29.00,44.0
rank_points,86230.0,1225.659886,1528.817899,1.0,510.0,793.0,1298.75,16790.0
rank,86230.0,77.518439,103.867787,1.0,26.0,54.0,92.00,2159.0
bpFaced,86230.0,6.175264,3.898824,0.0,3.0,6.0,9.00,29.0
bpSaved,86230.0,3.717511,2.917379,0.0,1.0,3.0,5.00,24.0
SvGms,86230.0,11.159364,2.948313,0.0,9.0,10.0,14.00,33.0
2ndWon,86230.0,14.443396,5.993341,0.0,10.0,14.0,18.00,60.0
1stWon,86230.0,30.665418,10.861418,0.0,23.0,29.0,37.00,101.0
1stIn,86230.0,43.081155,14.494993,0.0,32.0,41.0,52.00,135.0
svpt,86230.0,71.576655,21.793757,0.0,55.0,68.0,86.00,193.0


In [40]:
# Save the data to a new csv file
datapath = '../data'
save_file(df_all, 'tennis_data_all.csv',datapath)
save_file(df_numeric, 'tennis_data_numeric.csv',datapath)

Writing file.  "../data/tennis_data_all.csv"
Writing file.  "../data/tennis_data_numeric.csv"
